In [ ]:
#insecure eval

user_input=input("Enter expression: ")
result = eval(user_input)
print("Result: ", result)


# safe_eval_arith.py
import ast
import operator

# mapping AST operators to functions
operators = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow,
    ast.USub: operator.neg,
}

def eval_expr(node):
    if isinstance(node, ast.Num):  # < Py3.8
        return node.n
    if isinstance(node, ast.Constant):  # Python 3.8+
        return node.value
    if isinstance(node, ast.BinOp):
        left = eval_expr(node.left)
        right = eval_expr(node.right)
        op_type = type(node.op)
        if op_type in operators:
            return operators[op_type](left, right)
    if isinstance(node, ast.UnaryOp):
        operand = eval_expr(node.operand)
        op_type = type(node.op)
        if op_type in operators:
            return operators[op_type](operand)
    raise ValueError("Unsupported expression")

expr = input("Enter arithmetic expression: ")
parsed = ast.parse(expr, mode="eval")
result = eval_expr(parsed.body)
print("Result:", result)



Result:  5


In [10]:
# insecure_sql.py
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, pass TEXT)")
c.execute("INSERT INTO users (name, pass) VALUES ('alice', 'password123')")
c.execute("INSERT INTO users (name, pass) VALUES ('amrit', 'password456')")


username = input("Username: ")
query = "SELECT * FROM users WHERE name = '%s'" % username
c.execute(query)
print(c.fetchall())



# safe_sql.py
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, pass TEXT)")
c.execute("INSERT INTO users (name, pass) VALUES (?, ?)", ("alice", "password123"))

username = input("Username: ")
# the ? placeholder ensures parameters are bound safely
c.execute("SELECT * FROM users WHERE name = ?", (username,))
print(c.fetchall())



[(1, 'alice', 'password123'), (2, 'amrit', 'password456')]
[]


In [3]:
# insecure_cmd.py
import subprocess

filename = input("Enter filename to list: ")
# Dangerous: the Windows shell interprets special characters
subprocess.run(f"dir {filename}", shell=True)


# safe_cmd.py
import subprocess

filename = input("Enter filename to list: ")
# safe: pass args as a list and avoid shell=True
subprocess.run(["cmd", "/c", "dir", filename])


CompletedProcess(args=['cmd', '/c', 'dir', 'securee.ipynb & del D:\\Work\\Security\\5006\\mcp\\secure\\sample1.txt'], returncode=1)

In [ ]:
# insecure_pickle.py
import pickle

data = input("Paste pickled data (base64): ")
import base64
raw = base64.b64decode(data)
obj = pickle.loads(raw)  # Dangerous: executing untrusted pickle
print("Loaded object:", obj)



# safe_deser.py
import json

data = input("Paste JSON data: ")
obj = json.loads(data)
print("Loaded object:", obj)

In [ ]:
# insecure_upload.py (Flask)
from flask import Flask, request

app = Flask(__name__)

@app.route("/upload", methods=["POST"])
def upload():
    f = request.files['file']
    # Dangerous: directly using uploaded filename
    f.save("D:\\Work\\Security\\5006\\mcp\\secure" + f.filename)
    return "Saved"


# safe_upload.py (Flask)
from flask import Flask, request
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
UPLOAD_DIR = "/var/www/uploads"

@app.route("/upload", methods=["POST"])
def upload():
    f = request.files['file']
    filename = secure_filename(f.filename)  # sanitizes filename
    if not filename:
        return "Invalid filename", 400
    # ensure upload dir exists and join path safely
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    dest = os.path.join(UPLOAD_DIR, filename)
    f.save(dest)
    return "Saved"



In [ ]:
# insecure_password.py
password = input("Create password: ")
# Danger: saving plain text
with open("sample1.txt", "a") as f:
    f.write("alice:" + password + "\n")



# safe_password.py
import bcrypt

password = input("Create password: ").encode()
# generate salt and store hashed password
hashed = bcrypt.hashpw(password, bcrypt.gensalt())

# store hashed (binary) as base64 or decode to save as text
import base64
stored = base64.b64encode(hashed).decode()

# save stored to file or DB
with open("users.txt", "a") as f:
    f.write("alice:" + stored + "\n")

# to verify later
attempt = input("Enter password to check: ").encode()
# read stored and decode from base64 then verify
with open("users.txt") as f:
    lines = f.readlines()
stored = lines[-1].split(":",1)[1].strip()
hashed = base64.b64decode(stored)
if bcrypt.checkpw(attempt, hashed):
    print("Password OK")
else:
    print("Wrong password")



In [ ]:
# insecure_xss.py (Flask)
from flask import Flask, request
app = Flask(__name__)

@app.route("/greet", methods=["GET"])
def greet():
    name = request.args.get("name", "guest")
    return f"<h1>Hello {name}</h1>"


# safe_xss.py (Flask template)
from flask import Flask, request, render_template_string
app = Flask(__name__)

template = "<h1>Hello {{ name }}</h1>"

@app.route("/greet", methods=["GET"])
def greet():
    name = request.args.get("name", "guest")
    # render_template_string will auto-escape name
    return render_template_string(template, name=name)


Do not eval() untrusted input — parse safely.
Use parameterized queries for SQL.
Avoid shell=True and pass command args as lists.
Never unpickle untrusted data; use JSON or signed formats.
Sanitize filenames and prevent path traversal.
Hash passwords with bcrypt/argon2; never store plain text.
Keep secrets out of code; use env vars or secret managers.
Escape HTML output to prevent XSS.
Don’t log secrets.
Run services with least privilege.

In [ ]:
# unsafe_module.py - intentionally vulnerable
import os
import sqlite3
import subprocess

DB = "app.db"

def save_file(filename, content):
    # directly join user filename to upload dir
    path = "/var/www/uploads/" + filename
    with open(path, "w") as f:
        f.write(content)
    return path

def run_query(filter_value):
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    # unsafe: string formatting into SQL
    q = "SELECT * FROM logs WHERE message LIKE '%%%s%%'" % filter_value
    c.execute(q)
    return c.fetchall()

def system_list(dir_name):
    # runs shell command using user input
    cmd = "ls -la " + dir_name
    subprocess.run(cmd, shell=True)

def eval_expression(expr):
    # directly eval user-provided expression
    return eval(expr)


In [ ]:
# safe_module.py - safer input handling & validation
import os
import re
import sqlite3
import subprocess
from pathlib import Path
from werkzeug.utils import secure_filename
import shlex

DB = "app.db"
UPLOAD_DIR = "/var/www/uploads"
# whitelist patterns
FILENAME_RE = re.compile(r"^[A-Za-z0-9._-]{1,100}$")
ALPHANUM_RE = re.compile(r"^[A-Za-z0-9_-]{3,30}$")

def ensure_upload_dir():
    # create upload dir with safe permissions
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    os.chmod(UPLOAD_DIR, 0o750)

def validate_filename(name):
    """
    1) Use secure_filename to strip dangerous chars.
    2) Enforce a whitelist regex for allowed characters and length.
    """
    if not name:
        raise ValueError("Empty filename")
    safe = secure_filename(name)               # removes path separators, unicode tricks
    if not FILENAME_RE.match(safe):
        raise ValueError("Invalid filename pattern")
    return safe

def save_file_safe(filename, content):
    ensure_upload_dir()
    safe_name = validate_filename(filename)    # validate + sanitize
    dest = os.path.join(UPLOAD_DIR, safe_name)
    # canonicalize and ensure dest is inside UPLOAD_DIR
    dest_abs = os.path.abspath(dest)
    if not os.path.commonpath([os.path.abspath(UPLOAD_DIR), dest_abs]) == os.path.abspath(UPLOAD_DIR):
        raise ValueError("Path traversal attempt blocked")
    # write with explicit mode and safe encoding
    with open(dest_abs, "w", encoding="utf-8") as f:
        f.write(content)
    return dest_abs

def run_query_safe(filter_value):
    """
    1) Whitelist or escape filter inputs.
    2) Use parameterized queries with placeholders.
    For LIKE queries, construct the pattern safely.
    """
    if filter_value is None:
        raise ValueError("Filter value required")
    # small length boundary
    if len(filter_value) > 200:
        raise ValueError("Filter too long")
    # for logs we allow basic printable characters only
    if not re.match(r'^[\w\s\-\.,:;@#\(\)]*$', filter_value):
        raise ValueError("Invalid filter characters")
    conn = sqlite3.connect(DB)
    c = conn.cursor()
    # parameterized query; we add wildcards at Python-level (safe)
    pattern = f"%{filter_value}%"
    c.execute("SELECT * FROM logs WHERE message LIKE ?", (pattern,))
    rows = c.fetchall()
    conn.close()
    return rows

def system_list_safe(dir_name):
    """
    Avoid shell=True. Validate directory name and use list args.
    If a real shell is needed for expansion, use shlex.quote and a very strict whitelist.
    """
    if not dir_name:
        raise ValueError("Directory required")
    # avoid allowing metacharacters; allow relative simple names or absolute safe path under /var/www
    if ".." in dir_name or ";" in dir_name or "|" in dir_name:
        raise ValueError("Invalid directory name")
    # canonicalize
    p = Path(dir_name).resolve()
    # optional: enforce the directory is under an allowed base
    allowed_base = Path("/var/www")
    try:
        p.relative_to(allowed_base)
    except Exception:
        # not under /var/www: reject
        raise ValueError("Directory outside allowed area")
    # run ls with arguments (no shell)
    subprocess.run(["ls", "-la", str(p)], check=False)

def eval_expression_safe(expr):
    """
    Do not use eval on arbitrary strings. If expressions are strictly arithmetic,
    parse and evaluate only numbers/operators (use ast safer evaluator).
    """
    import ast
    import operator

    # allowed operators mapping
    operators = {ast.Add: operator.add, ast.Sub: operator.sub,
                 ast.Mult: operator.mul, ast.Div: operator.truediv,
                 ast.Pow: operator.pow, ast.USub: operator.neg}

    # small length boundary
    if len(expr) > 200:
        raise ValueError("Expression too long")

    def eval_node(node):
        if isinstance(node, ast.Constant):  # Python 3.8+
            if isinstance(node.value, (int, float)):
                return node.value
            raise ValueError("Only numeric constants allowed")
        if isinstance(node, ast.Num):  # older versions
            return node.n
        if isinstance(node, ast.BinOp):
            left = eval_node(node.left)
            right = eval_node(node.right)
            op_type = type(node.op)
            if op_type in operators:
                return operators[op_type](left, right)
            raise ValueError("Operator not allowed")
        if isinstance(node, ast.UnaryOp):
            operand = eval_node(node.operand)
            op_type = type(node.op)
            if op_type in operators:
                return operators[op_type](operand)
            raise ValueError("Operator not allowed")
        raise ValueError("Expression contains invalid nodes")

    parsed = ast.parse(expr, mode="eval")
    return eval_node(parsed.body)


In [8]:
#symmetric encryption 
# insecure_encrypt.py - bad example
from cryptography.fernet import Fernet

KEY = b"THIS_IS_INSECURE__DO_NOT_USE_BASE64_KEY"

def encrypt_message(msg):
    f = Fernet(KEY)
    return f.encrypt(msg.encode())

def decrypt_message(token):
    f = Fernet(KEY)
    return f.decrypt(token).decode()


In [ ]:
# secure_encrypt.py
import os
from cryptography.fernet import Fernet

def load_key():
    """
    Load encryption key from environment variable.
    If key does not exist, generate a new one (one-time setup).
    """
    key = os.getenv("FERNET_KEY")
    if key:
        return key.encode()

    # If no key exists, generate and instruct user to store it securely
    new_key = Fernet.generate_key()
    print("Generated new key. Store this securely as FERNET_KEY:")
    print(new_key.decode())
    raise RuntimeError("FERNET_KEY not set")

def encrypt_message(message):
    key = load_key()
    f = Fernet(key)
    token = f.encrypt(message.encode())
    return token

def decrypt_message(token):
    key = load_key()
    f = Fernet(key)
    return f.decrypt(token).decode()
